In [146]:
import numpy as np
import scipy.linalg as linalg
from dtuimldmtools import similarity
import matplotlib.pyplot as plt
from scipy.linalg import svd
from pandas.plotting import scatter_matrix
import pandas as pd
import numpy as np
import sklearn.linear_model as lm
from matplotlib.pyplot import figure, legend, plot, show, xlabel, ylabel
# exercise 8.1.1
from dtuimldmtools import dbplotf, train_neural_net, visualize_decision_boundary

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from collections import Counter
# from sklearn.neural_network import MLPRegress
import importlib_resources
import numpy as np
import sklearn.linear_model as lm
from matplotlib.pylab import (
    figure,
    grid,
    legend,
    loglog,
    semilogx,
    show,
    subplot,
    title,
    xlabel,
    ylabel,
)
from scipy.io import loadmat
from sklearn import model_selection

from dtuimldmtools import rlr_validate
import numpy as np
import scipy.linalg as linalg
from dtuimldmtools import similarity
import matplotlib.pyplot as plt
from scipy.linalg import svd
from pandas.plotting import scatter_matrix
import pandas as pd
import numpy as np
import sklearn.linear_model as lm
from matplotlib.pyplot import figure, legend, plot, show, xlabel, ylabel
# exercise 8.1.1
from dtuimldmtools import dbplotf, train_neural_net, visualize_decision_boundary
import importlib_resources
import numpy as np
import torch
from matplotlib.pyplot import figure, show, title
from scipy.io import loadmat
import importlib_resources
import numpy as np
import sklearn.linear_model as lm
from matplotlib.pylab import (
    figure,
    grid,
    legend,
    loglog,
    semilogx,
    show,
    subplot,
    title,
    xlabel,
    ylabel,
)
from scipy.io import loadmat
from sklearn import model_selection

from dtuimldmtools import rlr_validate

# fetch dataset 
# wine = fetch_ucirepo(id=109) 
  
# # data (as pandas dataframes) 
# X = wine.data.features 
# y = wine.data.targets 

# totaldata= (wine.data)
  
# metadata 
# print(wine.metadata) 
  
# variable information 
# print(wine.variables) 

# OFFLINE LOADING OF DATA
X = np.loadtxt('../wine/wine.data', delimiter=',')

#Extract classes from X
y = X[:,0]
X = np.delete(X,0,axis=1)
# y = np.loadtxt('../wine/wine.names', delimiter=',')

Xorig = X
# Standardizing the data

X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)



attributeNames = [
    "Alcohol",
    "Malic acid",
    "Ash",
    "Alcalinity of ash",
    "Magnesium",
    "Total phenols",
    "Flavanoids",
    "Nonflavanoid phenols",
    "Proanthocyanins",
    "Color intensity",
    "Hue",
    "OD280/OD315 of diluted wines",
    "Proline"
]

N, M = X.shape

# Add offset attribute
attributeNames = ["Offset"] + attributeNames
# M = M + 1


# Convert class labels to 0, 1, 2
y = y - 1

classNames = ["0", "1", "2"]


In [ ]:
# %% Model fitting and prediction using logistic regression

## Crossvalidation
# Create OUTER crossvalidation partition for evaluation
K = 10
CV = model_selection.KFold(K, shuffle=True, random_state=12)
# CV = model_selection.KFold(K, shuffle=False)

# Values of lambda
lambdas = np.logspace(0, 3, 100)
# Initialize variables
# T = len(lambdas)
Error_train = np.empty((K, 1))
Error_test = np.empty((K, 1))
Error_train_rlr = np.empty((K, 1))
Error_test_rlr = np.empty((K, 1))
Error_train_nofeatures = np.empty((K, 1))
Error_test_nofeatures = np.empty((K, 1))
w_rlr = np.empty((M, K))
mu = np.empty((K, M - 1))
sigma = np.empty((K, M - 1))
w_noreg = np.empty((M, K))
final_error_for_ANN = []
final_error_for_logreg = []
best_hidden_units = []
best_lambdas = []


k = 0
for train_index, test_index in CV.split(X, y):
    k += 1
    Error_train_avg = []
    Error_test_avg = []
    # extract training and test set for current CV fold
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    internal_cross_validation = 10
    
    internal_cv = model_selection.KFold(internal_cross_validation, shuffle=True, random_state=42)

    # Inner cross-validation to find the best regularization parameter (lambda)
    best_lambda = 0.0
    min_error = float('inf')
    for train_inner_idx, val_inner_idx in internal_cv.split(X_train, y_train):
        print("Outer Fold: ", k)
        #FOR LOGISTIC REGRESSION
        for l in lambdas:
            Error_train_rlr_fold = []
            Error_test_rlr_fold = []
            error_inner = []
        # For each lambda we do the training with internal cross validation
            X_train_inner, y_train_inner = X_train[train_inner_idx], y_train[train_inner_idx]
            
            X_val_inner, y_val_inner = X_train[val_inner_idx], y_train[val_inner_idx]

            logreg = lm.LogisticRegression(C=1/l, solver="lbfgs", tol=1e-4, random_state=1)
            logreg.fit(X_train_inner, y_train_inner)
            
            error_val = np.mean(logreg.predict(X_val_inner) != y_val_inner)

        # If the error is less than the minimum error, we update the best lambda
            # if error_val < min_error:
            #     min_error = error_val
            #     best_lambda = l
                
            error_train_new = np.mean(logreg.predict(X_train_inner) != y_train_inner)
            Error_train_rlr_fold.append(error_train_new)
            Error_test_rlr_fold.append(error_val)
            
        #_____________________________________________________
        # FOR ANN
        # Define the model structure
        max_hidden_units = 10
        error_log = []
        for n_hidden_units in range(5, max_hidden_units):
            print("Number of hidden units: ", n_hidden_units)
            # number of hidden units in the signle hidden layer
            C = 3
            model = lambda: torch.nn.Sequential(
                torch.nn.Linear(M, n_hidden_units),  # M features to H hiden units
                torch.nn.ReLU(),  # 1st transfer function
                # Output layer:
                # H hidden units to C classes
                # the nodes and their activation before the transfer
                # function is often referred to as logits/logit output
                torch.nn.Linear(n_hidden_units, C),  # C logits
                # To obtain normalised "probabilities" of each class
                # we use the softmax-funtion along the "class" dimension
                # (i.e. not the dimension describing observations)
                torch.nn.Softmax(dim=1),  # final tranfer function, normalisation of logit output
            )
            # Since we're training a multiclass problem, we cannot use binary cross entropy,
            # but instead use the general cross entropy loss:
            loss_fn = torch.nn.CrossEntropyLoss()
            # Train the network:
            # C = 3
            net, _, _ = train_neural_net(
                model,
                loss_fn,
                X=torch.tensor(X_train_inner, dtype=torch.float),
                y=torch.tensor(y_train_inner, dtype=torch.long),
                n_replicates=3,
                max_iter=3000
            )
            # Determine probability of each class using trained network
            softmax_logits = net(torch.tensor(X_val_inner, dtype=torch.float))
            # Get the estimated class as the class with highest probability (argmax on softmax_logits)
            y_test_est = (torch.max(softmax_logits, dim=1)[1]).data.numpy()
            # Determine errors
            e = y_test_est != y_val_inner
            # print(
            #     "Number of miss-classifications for ANN:\n\t {0} out of {1}".format(sum(e), len(e))
            # )
            error_log.append(np.mean(e))

            predict = lambda x: (
                torch.max(net(torch.tensor(x, dtype=torch.float)), dim=1)[1]
            ).data.numpy()
        #Find the best amount of hidden units
        # print(error_log)
        # print(range(1, max_hidden_units))
        # print(np.argmin(error_log))
        best_hidden_unit = range(1, max_hidden_units)[np.argmin(error_log)]
    
    #For ANN We just train with the best 
    #_____________________________________________________
    # Average errors across folds for each hidden unit
    
    C = 3
    model = lambda: torch.nn.Sequential(
        torch.nn.Linear(M, best_hidden_unit),  # M features to H hiden units
        torch.nn.ReLU(),  # 1st transfer function
        # Output layer:
        # H hidden units to C classes
        # the nodes and their activation before the transfer
        # function is often referred to as logits/logit output
        torch.nn.Linear(best_hidden_unit, C),  # C logits
        # To obtain normalised "probabilities" of each class
        # we use the softmax-funtion along the "class" dimension
        # (i.e. not the dimension describing observations)
        torch.nn.Softmax(dim=1),  # final tranfer function, normalisation of logit output
    )
    # Since we're training a multiclass problem, we cannot use binary cross entropy,
    # but instead use the general cross entropy loss:
    loss_fn = torch.nn.CrossEntropyLoss()
    # Train the network:
    # C = 3
    net, _, _ = train_neural_net(
        model,
        loss_fn,
        X=torch.tensor(X_train, dtype=torch.float),
        y=torch.tensor(y_train, dtype=torch.long),
        n_replicates=3,
        max_iter=3000
    )
    # Determine probability of each class using trained network
    softmax_logits = net(torch.tensor(X_test, dtype=torch.float))
    # Get the estimated class as the class with highest probability (argmax on softmax_logits)
    y_test_est = (torch.max(softmax_logits, dim=1)[1]).data.numpy()
    # Determine errors
    print(
        "Number of miss-classifications for ANN:\n\t {0} out of {1}".format(sum(y_test_est != y_test), len(y_test))
    )
    
    final_error_for_ANN.append(np.mean(y_test_est != y_test))
    predict = lambda x: (
        torch.max(net(torch.tensor(x, dtype=torch.float)), dim=1)[1]
    ).data.numpy()
    best_hidden_units.append(best_hidden_unit)
    print("Best hidden unit: ", best_hidden_unit)
    print("final ANN error for k=" + str(k) + " is " + str(np.mean(y_test_est != y_test)))
    
    
    
    #FOR LAMBDA / LOGISTIC REGRESSION
    #_____________________________________________________
    # Average errors across folds for each lambda
    Error_test_avg.append(np.mean(Error_test_rlr_fold))
    
    best_lambda = lambdas[np.argmin(Error_test_avg)]
    
    
    print("the best lambda is: ", best_lambda)
    
    
    # Train with best lambda on outer fold and evaluate amount of errors
    logreg_best = lm.LogisticRegression(C=best_lambda, solver="lbfgs",  tol=1e-4, random_state=1)
    logreg_best.fit(X_train, y_train)

    # Multinomial logistic regression
    logreg = lm.LogisticRegression(
        C=best_lambda,solver="lbfgs", multi_class="multinomial", tol=1e-4, random_state=1
    )
    logreg.fit(X_train, y_train)
    best_lambdas.append(best_lambda)
    final_error_for_logreg.append(np.mean(logreg.predict(X_test) != y_test))
    print("final logreg error for k=" + str(k) + " is " + str(np.mean(logreg.predict(X_test) != y_test)))

    #_____________________________________________________
    
    #For the baseline
    #_____________________________________________________
    most_common_class = Counter(y_train).most_common(1)[0][0]

    # Predict the most common class for all test and train samples
    y_est_train_base = np.full(len(y_train), most_common_class)
    y_est_test_base = np.full(len(y_test), most_common_class)

    # Evaluate misclassification rate for train and test sets
    misclass_rate_train = np.sum(y_est_train_base != y_train) / len(y_train)
    misclass_rate_test = np.sum(y_est_test_base != y_test) / len(y_test)


    # Store the errors
    Error_train_nofeatures[k-1] = misclass_rate_train
    Error_test_nofeatures[k-1] = misclass_rate_test

    print("   Top class: ", most_common_class+1, "  Test error: ", round(misclass_rate_test,3))
    
    #_____________________________________________________
    
    
print("final_error_for_ANN: ", (final_error_for_ANN))
print("best_hidden_units: ", (best_hidden_units))
print("final_error_for_logreg: ", (final_error_for_logreg))
print("best_lambdas: ", (best_lambdas))
print("final_error_for_baseline: ", (Error_test_nofeatures))
# Create a matrix from the arrays
# matrix = np.column_stack((Error_test, Error_test_avg, Error_test_nofeatures, Error_test_rlr, Error_test_rlr_fold, Error_train, Error_train_avg, Error_train_nofeatures, Error_train_rlr, Error_train_rlr_fold))

# Output the matrix to the terminal

# Save the matrix to a txt file
# np.savetxt('output_matrix.txt', matrix, delimiter=',', fmt='%s')











# To display coefficients use print(logreg.coef_). For a 4 class problem with a
# feature space, these weights will have shape (4, 2).

# Number of miss-classifications
# print(
#     "Number of miss-classifications for Multinormal regression:\n\t {0} out of {1}".format(
#         np.sum(logreg.predict(X_test) != y_test), len(y_test)
#     )
# )
# print(logreg.predict(X_test))
# print(y_test)

# print("Coefficients")
# print(logreg.coef_)
# # Plotting should be done after the outer loop
# plt.figure(figsize=(10, 6))
# plt.semilogx(lambdas, Error_train_avg, 'b-', label='Training Error')
# plt.semilogx(lambdas, Error_test_avg, 'r-', label='Test Error')
# plt.xlabel('Lambda')
# plt.ylabel('Error')
# plt.legend()
# plt.title('Training vs Test Error for Different Lambda Values')
# plt.grid()
# plt.show()
# k += 1
# print("final_error_for_ANN: ", (final_error_for_ANN))
# print("final_error_for_logreg: ", (final_error_for_logreg))


Outer Fold:  1
Number of hidden units:  1

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.8586534	6.434488e-05
		2000	0.8163155	3.1469197e-05
		3000	0.8008	1.0866862e-05
		Final loss:
		3000	0.8008	1.0866862e-05

	Replicate: 2/3
		Iter	Loss			Rel. loss


/Users/lauge/Desktop/ML/MLenv/lib/python3.12/site-packages/dtuimldmtools/models/nn_trainer.py:141: RuntimeWarning: overflow encountered in cast
  if loss_value < best_final_loss:


		1000	0.8350412	0.000108342014
		2000	0.79656476	2.3495162e-05
		3000	0.7813111	1.441822e-05
		Final loss:
		3000	0.7813111	1.441822e-05

	Replicate: 3/3
		Iter	Loss			Rel. loss
		1000	0.8634782	4.0932253e-05
		2000	0.8342418	3.0650117e-05
		3000	0.72802216	0.0001112517
		Final loss:
		3000	0.72802216	0.0001112517
Number of hidden units:  2

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.85447514	0.00010378594
		2000	0.5939797	0.00010545457
		3000	0.56598127	2.1693812e-05
		Final loss:
		3000	0.56598127	2.1693812e-05

	Replicate: 2/3
		Iter	Loss			Rel. loss
		1000	0.6313383	0.00030400214
		2000	0.57209134	3.542242e-05
		3000	0.56040215	1.1593158e-05
		Final loss:
		3000	0.56040215	1.1593158e-05

	Replicate: 3/3
		Iter	Loss			Rel. loss
		1000	0.6661591	0.00020422992
		2000	0.5756679	4.1517807e-05
		3000	0.5619349	1.3470756e-05
		Final loss:
		3000	0.5619349	1.3470756e-05
Outer Fold:  1
Number of hidden units:  1

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.8798399	4.5116052e-05


/Users/lauge/Desktop/ML/MLenv/lib/python3.12/site-packages/dtuimldmtools/models/nn_trainer.py:141: RuntimeWarning: overflow encountered in cast
  if loss_value < best_final_loss:


		1000	0.8158677	0.0001235237
		2000	0.77119315	2.465454e-05
		3000	0.75931776	1.0440076e-05
		Final loss:
		3000	0.75931776	1.0440076e-05

	Replicate: 3/3
		Iter	Loss			Rel. loss
		1000	0.8099911	0.00012324267
		2000	0.7571356	5.52611e-05
		3000	0.68988216	6.203013e-05
		Final loss:
		3000	0.68988216	6.203013e-05
Number of hidden units:  2

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.79998	7.4576616e-05
		2000	0.71927106	0.001118131
		3000	0.5601281	1.7238532e-05
		Final loss:
		3000	0.5601281	1.7238532e-05

	Replicate: 2/3
		Iter	Loss			Rel. loss
		1000	0.6372131	0.00020630573
		2000	0.5699174	3.493007e-05
		3000	0.55906796	9.381971e-06
		Final loss:
		3000	0.55906796	9.381971e-06

	Replicate: 3/3
		Iter	Loss			Rel. loss
		1000	0.62872934	0.00015042775
		2000	0.58558553	2.9313625e-05
		3000	0.5758633	9.108344e-06
		Final loss:
		3000	0.5758633	9.108344e-06

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.6015245	0.00012988922
		2000	0.5684579	1.8139279e-05
		3000	0.5628223	5.29

...

/Users/lauge/Desktop/ML/MLenv/lib/python3.12/site-packages/dtuimldmtools/models/nn_trainer.py:141: RuntimeWarning: overflow encountered in cast
  if loss_value < best_final_loss:


		1000	0.8662357	7.9467856e-05
		2000	0.82907283	2.7462424e-05
		3000	0.80730915	1.9934041e-05
		Final loss:
		3000	0.80730915	1.9934041e-05

	Replicate: 3/3
		Iter	Loss			Rel. loss
		1000	0.8238228	4.9268805e-05
		2000	0.7905402	0.00010614821
		3000	0.7486977	2.1812964e-05
		Final loss:
		3000	0.7486977	2.1812964e-05
Number of hidden units:  2

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.60199344	0.00015908715
		2000	0.5662376	2.4210218e-05
		3000	0.5581192	8.4367775e-06
		Final loss:
		3000	0.5581192	8.4367775e-06

	Replicate: 2/3
		Iter	Loss			Rel. loss
		1000	0.6408215	0.00022504048
		2000	0.57599026	4.283973e-05
		3000	0.5618867	1.36840645e-05
		Final loss:
		3000	0.5618867	1.36840645e-05

	Replicate: 3/3
		Iter	Loss			Rel. loss
		1000	0.6006616	0.00014118671
		2000	0.56562096	2.3815117e-05
		3000	0.55773455	8.015126e-06
		Final loss:
		3000	0.55773455	8.015126e-06
Outer Fold:  3
Number of hidden units:  1

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.9199864	0.0001146627

/Users/lauge/Desktop/ML/MLenv/lib/python3.12/site-packages/dtuimldmtools/models/nn_trainer.py:141: RuntimeWarning: overflow encountered in cast
  if loss_value < best_final_loss:


		1000	0.823382	7.433903e-05
		2000	0.77583545	6.7064946e-05
		3000	0.7289394	5.0530685e-05
		Final loss:
		3000	0.7289394	5.0530685e-05

	Replicate: 3/3
		Iter	Loss			Rel. loss
		1000	0.78150874	5.9257258e-05
		2000	0.7316156	7.144399e-05
		3000	0.69330245	3.7826278e-05
		Final loss:
		3000	0.69330245	3.7826278e-05
Number of hidden units:  2

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.62324816	0.00018970482
		2000	0.5698957	3.3258097e-05
		3000	0.5592835	1.0230925e-05
		Final loss:
		3000	0.5592835	1.0230925e-05

	Replicate: 2/3
		Iter	Loss			Rel. loss
		1000	0.63610375	0.00020432385
		2000	0.5775026	4.2211534e-05
		3000	0.5630955	1.4501479e-05
		Final loss:
		3000	0.5630955	1.4501479e-05

	Replicate: 3/3
		Iter	Loss			Rel. loss
		1000	0.67448425	0.00035388774
		2000	0.5827593	5.8193953e-05
		3000	0.56403524	1.7330463e-05
		Final loss:
		3000	0.56403524	1.7330463e-05

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.630784	0.00018649432
		2000	0.57439953	3.9638042e-05
		3000	0.5

/Users/lauge/Desktop/ML/MLenv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
